In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_date, to_date, when
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, BooleanType
from datetime import date

# Initialize Spark session
spark = SparkSession.builder.appName("SCD_Type2_LeftJoin_Example").getOrCreate()

# Define schema for the existing customer dimension table (target)
customer_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("email", StringType(), True),
    StructField("effective_date", DateType(), True),
    StructField("end_date", DateType(), True),
    StructField("is_current", BooleanType(), True)
])

# Sample data for existing customer dimension table using Python date objects
existing_data = [
    (1, "John Doe", "123 Old St", "john@example.com", date(2023, 1, 1), None, True),
    (2, "Jane Smith", "456 Old Ave", "jane@example.com", date(2023, 1, 1), None, True)
]

# Create DataFrame for existing customer table
customer_df = spark.createDataFrame(existing_data, customer_schema)
print("customer_df")
customer_df.display()
# Define schema for incoming updates (source)
update_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("email", StringType(), True),
    StructField("update_date", DateType(), True)
])

# Sample incoming data with updates using Python date objects
update_data = [
    (1, "John Doe", "789 New St", "john.doe@example.com", date(2023, 6, 1)),  # Address and email changed
    (2, "Jane Smith", "456 Old Ave", "jane@example.com", date(2023, 6, 1)),   # No change
    (3, "Bob Wilson", "101 New Rd", "bob@example.com", date(2023, 6, 1))      # New customer
]

# Create DataFrame for incoming updates
updates_df = spark.createDataFrame(update_data, update_schema)
print("updates_df")
updates_df.display()



In [0]:
joined_df = updates_df.join(
    customer_df,
    (updates_df.customer_id == customer_df.customer_id) & (customer_df.is_current == True),
    "left_outer"
)
print("joined_df")
joined_df.display()

filtered_df = joined_df.filter(
    # New records (no match in customer_df) or changed records
    (customer_df.customer_id.isNull()) |
    (updates_df.address != customer_df.address) |
    (updates_df.email != customer_df.email)
).select(
    updates_df.customer_id,
    updates_df.name,
    updates_df.address,
    updates_df.email,
    updates_df.update_date.alias("effective_date"),
    lit(None).cast(DateType()).alias("end_date"),
    lit(True).alias("is_current")
)

filtered_df.display()

In [0]:
expired_data = joined_df.filter((customer_df.customer_id.isNotNull()) &
    ((updates_df.address != customer_df.address) | (updates_df.email != customer_df.email)))\
    .select(
    customer_df.customer_id,
    customer_df.name,
    customer_df.address,
    customer_df.email,
    customer_df.effective_date,
    updates_df.update_date.alias("end_date"),
    lit(False).alias("is_current"))
expired_data.display()

In [0]:
unchanged_records = customer_df.join(
    filtered_df,
    customer_df.customer_id == updates_df.customer_id,
    "left_anti"
).select(
    customer_df.customer_id,
    customer_df.name,
    customer_df.address,
    customer_df.email,
    customer_df.effective_date,
    customer_df.end_date,
    customer_df.is_current
)
unchanged_records.display()


In [0]:
final_df = filtered_df.union(expired_data).union(unchanged_records)
final_df.display()
final_df.orderBy("customer_id", "effective_date").display()

### Source DataFrame (latest snapshot from source system)

In [0]:
new_source_data = [
    (101, "Alice Smith", "Chicago"),  # City changed
    (102, "Bob Brown", "Seattle")     # No change
]
df_new_source = spark.createDataFrame(new_source_data, ["CustomerID", "Name", "City"])
df_new_source.display()

###  Target Dimension DataFrame (Customer_Dim)

In [0]:
from pyspark.sql.functions import current_date, lit

# Initial load - simulate source
initial_source_data = [
    (101, "Alice Smith", "Boston"),
    (102, "Bob Brown", "Seattle")
]

df_initial_source = spark.createDataFrame(initial_source_data, ["CustomerID", "Name", "City"])

# Add SCD Type 2 columns
df_initial_target = df_initial_source\
    .withColumn("StartDate", current_date()) \
    .withColumn("EndDate", lit("9999-12-31").cast("date")) \
    .withColumn("IsCurrent", lit(True))

# Reorder columns
df_initial_target = df_initial_target.select(
    "CustomerID", "Name", "City", "StartDate", "EndDate", "IsCurrent"
)

df_initial_target.display()


In [0]:
joined_df = df_new_source.join(df_initial_target, (df_new_source.CustomerID == df_initial_target.CustomerID) & (df_initial_target.IsCurrent == True), "left_outer")
joined_df.display()

changed_df = joined_df.filter((df_initial_target.CustomerID.isNull()) | (df_new_source.City != df_initial_target.City) | (df_new_source.Name != df_initial_target.Name)).select(
    df_new_source.CustomerID,
    df_new_source.Name,
    df_new_source.City,
    date_add(current_date(),1).alias("StartDate"),
    lit("9999-12-31").cast("date").alias("EndDate"),
    lit(True).alias("IsCurrent")
)
changed_df.display()


In [0]:
from pyspark.sql.functions import current_date, lit, date_sub, date_add



In [0]:
expired_df = joined_df.filter((df_initial_target.CustomerID.isNotNull()) & (df_new_source.City != df_initial_target.City) | (df_new_source.Name != df_initial_target.Name))
expired_df.display()

expired_df = expired_df.select(
    df_initial_target.CustomerID,
    df_initial_target.Name,
    df_initial_target.City,
    df_initial_target.StartDate,
    current_date().alias("EndDate"),
    lit(False).alias("IsCurrent")
)
expired_df.display()

In [0]:
unchanged_records = df_initial_target.join(changed_df, "CustomerID", "left_anti")
unchanged_records.display()

final_df = unchanged_records.unionByName(changed_df).unionByName(expired_df)
final_df.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, BooleanType
from datetime import date

spark = SparkSession.builder.getOrCreate()

# Initial dimension data with historical StartDate
initial_data = [
    (1, 1001, "iPhone 14", "Mobile", 999.0, date(2024, 1, 1), date(9999, 12, 31), True),
    (2, 1002, "Galaxy S23", "Mobile", 849.0, date(2024, 1, 1), date(9999, 12, 31), True),
    (3, 1003, "Dell XPS 13", "Laptop", 1199.0, date(2024, 1, 1), date(9999, 12, 31), True)
]

schema = StructType([
    StructField("SurrogateKey", IntegerType(), False),
    StructField("ProductID", IntegerType(), False),
    StructField("ProductName", StringType(), False),
    StructField("Category", StringType(), False),
    StructField("Price", DoubleType(), False),
    StructField("StartDate", DateType(), False),
    StructField("EndDate", DateType(), False),
    StructField("IsCurrent", BooleanType(), False)
])

df_dim_initial = spark.createDataFrame(initial_data, schema)
df_dim_initial.display()


In [0]:
# New snapshot from source (today's data)
new_products = [
    (1001, "iPhone 14", "Mobile", 999.0),       # No change
    (1002, "Galaxy S23", "Mobile", 899.0),      # Price updated
    (1003, "Dell XPS 13", "Laptop", 1199.0)     # No change
]

df_source = spark.createDataFrame(new_products, ["ProductID", "ProductName", "Category", "Price"])
df_source.display()


In [0]:
join_condition = (
    (df_source.ProductID  ==  df_dim_initial.ProductID) & (df_dim_initial.IsCurrent == True)
)
joined_df  = df_source.join(df_dim_initial, join_condition
                            , "left_outer")
joined_df.display()

changed_df = joined_df.filter((df_dim_initial.ProductID.isNull()) | (df_source.Price != df_dim_initial.Price))
changed_df.display()

changed_df = changed_df.select(
    df_source.ProductID.alias("ProductID"),
    df_source.ProductName.alias("ProductName"),
    df_source.Category.alias("Category"),
    df_source.Price.alias("Price"),
    date_add(current_date(),1).alias("StartDate"),
    lit("9999-12-31").cast("date").alias("EndDate"),
    lit(True).alias("IsCurrent")
)
changed_df.display()

In [0]:
expired_df = joined_df.filter((df_dim_initial.ProductID.isNotNull()) & (df_source.Price != df_dim_initial.Price))
expired_df.display()
expired_df = expired_df.select(
    df_dim_initial.ProductID.alias("ProductID"),
    df_dim_initial.ProductName.alias("ProductName"),
    df_dim_initial.Category.alias("Category"),
    df_dim_initial.Price.alias("Price"),
    df_dim_initial.StartDate.alias("StartDate"),
    current_date().alias("EndDate"),
    lit(False).alias("IsCurrent")
)
expired_df.display()

In [0]:
unchanged_records = df_dim_initial.join(changed_df, "ProductID", "left_anti").drop("SurrogateKey")
unchanged_records.display()
final_df = unchanged_records.unionByName(changed_df).unionByName(expired_df)
final_df.display()

Using Merge

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType

# Create a Spark session (should already exist in Databricks)
spark = SparkSession.builder.getOrCreate()

# Initial customer data (Day 1)
initial_data = [
    (1, "Alice", "New York"),
    (2, "Bob", "San Francisco"),
    (3, "Charlie", "Chicago")
]

schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("city", StringType(), True)
])

df_initial = spark.createDataFrame(initial_data, schema)

# Add SCD2 columns
df_initial = df_initial.withColumn("start_date", current_date()) \
    .withColumn("end_date", lit("9999-12-31")) \
    .withColumn("is_current", lit(True))
df_initial.display()

# Save as Delta table
df_initial.write.format("delta").mode("overwrite").saveAsTable("incremental_load.default.dim_customers_scd2")


In [0]:
# Day 2 source data with changes
day2_data = [
    (1, "Alice", "New York"),           # unchanged
    (2, "Bob", "Los Angeles"),          # changed city
    (4, "David", "Houston")             # new customer
]

df_day2 = spark.createDataFrame(day2_data, schema)

# Add SCD2 columns to incoming data
df_day2 = df_day2.withColumn("start_date", current_date()) \
    .withColumn("end_date", lit("9999-12-31")) \
    .withColumn("is_current", lit(True))
df_day2.display()

# Register as temp view for SQL-based MERGE
df_day2.createOrReplaceTempView("staging_customers")



In [0]:
%sql
MERGE INTO incremental_load.default.dim_customers_scd2 AS target
USING staging_customers AS source
ON target.customer_id = source.customer_id AND target.is_current = true

WHEN MATCHED AND target.city <> source.city THEN
  UPDATE SET target.end_date = '2024-05-13', target.is_current = false

WHEN NOT MATCHED THEN
  INSERT (customer_id, name, city, start_date, end_date, is_current)
  VALUES (source.customer_id, source.name, source.city, source.start_date, source.end_date, source.is_current)


In [0]:
spark.sql("SELECT * FROM incremental_load.default.dim_customers_scd2 ORDER BY customer_id, start_date").show(truncate=False)


In [0]:
%sql
SELECT * 
FROM incremental_load.default.dim_customers_scd2 
WHERE customer_id = 2 
ORDER BY start_date;


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_date, when
from datetime import date
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

spark = SparkSession.builder.appName("SCD Type 2 with datetime.date").getOrCreate()

# Step 1: Simulated source data (incoming data)
source_data = [
    (1, "Alice", "USA"),     # Unchanged
    (2, "Bob", "Canada"),    # Updated (was UK)
    (4, "Daisy", "India"),   # New record
]
source_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("country", StringType(), False),
])
source_df = spark.createDataFrame(source_data, schema=source_schema)

# Step 2: Existing dimension table (dim_customer)
# Use datetime.date to create valid dates in dim_data
dim_data = [
    (1, "Alice", "USA", date(2020, 1, 1), None),  # Full date (year-month-day)
    (2, "Bob", "UK", date(2019, 1, 1), None),     # Full date (year-month-day)
    (3, "Charlie", "Germany", date(2021, 1, 1), None),  # Full date (year-month-day)
]
dim_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("country", StringType(), False),
    StructField("start_date", DateType(), False),
    StructField("end_date", DateType(), True),
])

# Create the DataFrame using the datetime.date objects in dim_data
dim_df = spark.createDataFrame(dim_data, schema=dim_schema)

# Step 3: Perform SCD Type 2 Merge - Maintain History

# Add today's date for merge logic
today = date.today()  # Today's date using datetime.date()

# Step 4: Prepare source data to include start_date and end_date columns
source_with_dates = source_df.withColumn("start_date", lit(today)).withColumn("end_date", lit(None).cast("date"))

# Create temp view for SQL merge
source_with_dates.createOrReplaceTempView("source_customer")

# Perform Merge Logic
spark.sql(f"""
MERGE INTO incremental_load.default.dim_customer_v2 AS target
USING source_customer AS source
ON target.customer_id = source.customer_id AND target.end_date IS NULL

-- When a match is found and data has changed, close the current record
WHEN MATCHED AND (
    target.name <> source.name OR
    target.country <> source.country
) THEN
  UPDATE SET target.end_date = current_date()

-- When no match is found, insert a new record
WHEN NOT MATCHED THEN
  INSERT (customer_id, name, country, start_date, end_date)
  VALUES (source.customer_id, source.name, source.country, source.start_date, source.end_date)
""")

# Show the resulting dimension table to see the changes
result_df = spark.sql("SELECT * FROM incremental_load.default.dim_customer_v2")
result_df.show(truncate=False)


In [0]:
%sql
select * from incremental_load.default.dim_customer_v2

bank Example


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import date

# Initialize Spark session
spark = SparkSession.builder.master("local").appName("SCD Type 2 Example").getOrCreate()

# Existing dim_bank_account (historical data)
dim_data = [
    (1001, 1, 1000, "Active", date(2020, 1, 1), None),  # Account 1001 is active since 2020-01-01
    (1002, 2, 1500, "Active", date(2020, 1, 1), None),  # Account 1002 is active since 2020-01-01
    (1003, 3, 500, "Active", date(2021, 1, 1), None),   # Account 1003 is active since 2021-01-01
]
dim_schema = StructType([
    StructField("account_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("account_balance", IntegerType(), False),
    StructField("account_status", StringType(), False),
    StructField("start_date", DateType(), False),
    StructField("end_date", DateType(), True),
])

dim_df = spark.createDataFrame(dim_data, schema=dim_schema)
dim_df.show()


In [0]:
# Incoming data for source_bank_account
source_data = [
    (1001, 1, 1200, "Active"),  # Account 1001 balance updated
    (1002, 2, 1500, "Closed"),  # Account 1002 status changed from Active to Closed
    (1004, 4, 2000, "Active"),  # New account (1004)
]
source_schema = StructType([
    StructField("account_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("account_balance", IntegerType(), False),
    StructField("account_status", StringType(), False),
])

source_df = spark.createDataFrame(source_data, schema=source_schema)
source_df.show()


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import date

# Initialize Spark session
spark = SparkSession.builder.master("local").appName("SCD Type 2 Example").getOrCreate()

# Existing dim_bank_account (historical data)
dim_data = [
    (1001, 1, 1000, "Active", date(2020, 1, 1), None),  # Account 1001 is active since 2020-01-01
    (1002, 2, 1500, "Active", date(2020, 1, 1), None),  # Account 1002 is active since 2020-01-01
    (1003, 3, 500, "Active", date(2021, 1, 1), None),   # Account 1003 is active since 2021-01-01
]
dim_schema = StructType([
    StructField("account_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("account_balance", IntegerType(), False),
    StructField("account_status", StringType(), False),
    StructField("start_date", DateType(), False),
    StructField("end_date", DateType(), True),
])

dim_df = spark.createDataFrame(dim_data, schema=dim_schema)
dim_df.write.format("delta").mode("overwrite").saveAsTable("incremental_load.default.dim_bank_account")
# 

dim_df.show()


In [0]:
# Incoming data for source_bank_account
source_data = [
    (1004, 1, 3000, "Active"),  # Account 1001 balance updated
    (1002, 2, 1500, "Closed"),  # Account 1002 status changed from Active to Closed
    (1004, 4, 2000, "Active"),  # New account (1004)
]
source_schema = StructType([
    StructField("account_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("account_balance", IntegerType(), False),
    StructField("account_status", StringType(), False),
])

source_df = spark.createDataFrame(source_data, schema=source_schema)
source_df.show()

source_df.write.format("delta").mode("overwrite").saveAsTable("incremental_load.default.source_bank_account")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, lit

# Initialize Spark session (if not already initialized)
spark = SparkSession.builder.appName("SCD_Type2_Example").getOrCreate()

# Step 1: Perform the MERGE operation for SCD Type 2 using `incremental_load.default` catalog and schema
spark.sql("""
MERGE INTO incremental_load.default.dim_bank_account AS target
USING incremental_load.default.source_bank_account AS source
ON target.account_id = source.account_id AND target.end_date IS NULL

-- When a match is found and data has changed, close the current record (end_date)
WHEN MATCHED AND (
    target.account_status <> source.account_status OR
    target.account_balance <> source.account_balance
) THEN
  UPDATE SET 
  target.account_status = 'closed',
  target.end_date = current_date()
    -- Mark the current record as historical (end_date)

-- When no match is found, insert a new record (new account or new entry)
WHEN NOT MATCHED THEN
  INSERT (account_id, customer_id, account_balance, account_status, start_date, end_date)
  VALUES (source.account_id, source.customer_id, source.account_balance, source.account_status, current_date(), NULL)  -- New record is active with NULL end_date
""")


In [0]:
%sql
select * from incremental_load.default.dim_bank_account

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_date
from datetime import date

schema = StructType([
    StructField("bank_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("start_date", DateType(), True),
    StructField("end_date", DateType(), True)  # Can be null
])

# Sample new data coming in
new_data = spark.createDataFrame([
    (1, "ABC Bank", "New York", date(2020, 1, 1), None),    # No end_date → active
    (2, "XYZ Bank", "Chicago", date(2020, 5, 1), None),  # Has end_date → closed
    (3, "New Bank", "Houston", date(2021, 6, 1), None)      # New bank, no history
], schema) \
    .withColumn("status", lit("active"))  # Default, will be updated

# Update status based on end_date
new_data.display()


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import to_date

# Define schema
schema = StructType([
    StructField("bank_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("start_date", StringType(), True),  # Temporarily as string
    StructField("end_date", StringType(), True),
    StructField("status", StringType(), True)
])

# Data (dates still as strings here)
data = [
    (1, "ABC Bank", "New York", "2020-01-01", "9999-12-31", "active"),
    (2, "XYZ Bank", "Chicago", "2020-05-01", "9999-12-31", "active")
]

# Create DataFrame
history_data = spark.createDataFrame(data, schema=schema)

# Convert date columns to proper DateType
history_data = history_data.withColumn("start_date", to_date("start_date")) \
                           .withColumn("end_date", to_date("end_date"))
history_data.display()

# Save to Delta
history_data.write.format("delta").mode("overwrite").saveAsTable("incremental_load.default.banks_history")


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import to_date, lit, when
updated_data = [
    (1, "ABC Bank", "Boston", "2024-01-01", None),           # Changed location
    (2, "XYZ Bank", "Chicago", "2020-05-01", "2023-12-31"),  # Closed bank
    (3, "New Bank", "Dallas", "2024-02-15", None)            # New record
]

# Define schema
update_schema = StructType([
    StructField("bank_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("start_date", StringType(), True),
    StructField("end_date", StringType(), True)
])

# Create DataFrame
df_updates = spark.createDataFrame(updated_data, schema=update_schema)

# Convert dates to proper format
df_updates = df_updates.withColumn("start_date", to_date("start_date")) \
                       .withColumn("end_date", to_date("end_date"))

# Add derived status column based on end_date
df_updates = df_updates.withColumn(
    "status",
    when(df_updates["end_date"].isNotNull(), "closed").otherwise("active")
)

df_updates.display()


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date

# Load Delta Table
delta_table = DeltaTable.forName(spark, "incremental_load.default.banks_history")

# SCD Type 2 Merge Logic
delta_table.alias("target").merge(
    df_updates.alias("source"),
    "target.bank_id = source.bank_id AND target.end_date = '9999-12-31'"
).whenMatchedUpdate(condition="""
    target.name != source.name OR
    target.location != source.location OR
    target.status != source.status
""", set={
    "end_date": lit(date.today().isoformat()),  # Close the old record
    "status": lit("closed")
}).whenNotMatchedInsert(values={
    "bank_id": "source.bank_id",
    "name": "source.name",
    "location": "source.location",
    "start_date": lit(date.today().isoformat()),
    "end_date": lit("9999-12-31"),
    "status": "source.status"
}).execute()

# delta_table.write.format("delta").mode("overwrite").saveAsTable("incremental_load.default.banks_history")

In [0]:
spark.read.format("delta").table("incremental_load.default.banks_history").show(truncate=False)


In [0]:
%sql
drop table incremental_load.default.credit_cards

In [0]:
from pyspark.sql.functions import current_date, lit, expr
from delta.tables import DeltaTable
from datetime import date

delta_table_target = "incremental_load.default.dim_customer4"

initial_data = [
    (101, "Alice", "New York", date(2024, 1, 1), None,True),
    (102, "Bob", "San Francisco", date(2024, 1, 1), None,True),
]

schema = StructType([
    StructField("Customer_ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Start_Date", DateType(), True),
    StructField("End_Date", DateType(), True),
    StructField("IsCurrent", BooleanType(), True)
])

df_initial = spark.createDataFrame(initial_data, schema)

df_initial.write.format("delta").mode("overwrite").saveAsTable(delta_table_target)

In [0]:
incoming_data = [
    (101, "Alice", "New York"),
    (102, "Bob", "Los Angeles"),
    (103, "Charlie", "Chicago")
]

schema = StructType([
    StructField("Customer_ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("City", StringType(), True)
])

df_incoming = spark.createDataFrame(incoming_data, schema)
df_incoming = df_incoming.withColumn("Start_Date", current_date()) \
        .withColumn("End_Date", lit("9999-12-31")) \
        .withColumn("IsCurrent", lit(True))

delta_table = DeltaTable.forName(spark, delta_table_target)

delta_table.alias("target").merge(
    source=df_incoming.alias("source"),
    condition="target.Customer_ID = source.Customer_ID AND target.IsCurrent = True"
).whenMatchedUpdate(
    condition="""
        target.Name != source.Name OR  
        target.City != source.City OR  
    """,
    set={
        "End_Date": expr("current_date()"),
        "IsCurrent": lit(False)
    }

).whenNotMatchedInsert(
    values={
        "Customer_ID": "source.Customer_ID",
        "Name": "source.Name",
        "City": "source.City",
        "Start_Date": expr("current_date()"),
        "End_Date": lit("9999-12-31"),
        "IsCurrent": lit(True)
}).execute()


spark.read.format("delta").table(delta_table_target).display()


In [0]:
delta_table_path = "incremental_load.default.dim_credit_card_customers"

initial_data = [
    (101, "John Doe", "1234 Em st", "Platinum", date(2021, 1, 1), None, True),
    (102, "Jane Smith", "4567 Oak st", "Gold", date(2021, 1, 1), None, True)
]

schema = StructType([
    StructField("Customer_ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("Card_Type", StringType(), True),
    StructField("Start_Date", DateType(), True),
    StructField("End_Date", DateType(), True),
    StructField("IsCurrent", BooleanType(), True)
])

df_initial = spark.createDataFrame(initial_data, schema)

df_initial.write.format("delta").mode("overwrite").saveAsTable(delta_table_path)

In [0]:
incoming_data = [
    (101, "John Doe", "1234 Main st", "Diamond"),
    (102, "Jane Smith", "888* Maple Ave", "Gold"),
    (103, "Alice Johnson", "9999 Pine st", "Platinum")

]

schema = StructType([
    StructField("Customer_ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("Card_Type", StringType(), True),
])

df_incoming = spark.createDataFrame(incoming_data, schema)

df_incoming = df_incoming.withColumn("Start_Date", current_date()) \
        .withColumn("End_Date",lit(None))\
        .withColumn("IsCurrent",lit(True))

delta_table = DeltaTable.forName(spark, delta_table_path)

delta_table.alias("target").merge(
    source=df_incoming.alias("source"),
    condition="""
        target.Customer_ID = source.Customer_ID AND target.IsCurrent = True
    """

).whenMatchedUpdate(
    condition="""
        target.Name != source.Name OR  
        target.Address != source.Address OR
        target.Card_Type != source.Card_Type
        """,
    set = {
        "End_Date": expr("current_date()"),
        "IsCurrent": lit(False)
    }
).whenNotMatchedInsert(
    values={
        "Customer_ID": "source.Customer_ID",
        "Name": "source.Name",
        "Address": "source.Address",
        "Card_Type": "source.Card_Type",
        "Start_Date": expr("current_date()"),
        "End_Date": lit(None),
        "IsCurrent": lit(True)
    }
).execute()


In [0]:
spark.read.format("delta").table(delta_table_path).display()
